# argparse --- 命令行选项、参数和子命令解析器
[https://docs.python.org/zh-cn/3.8/library/argparse.html#module-argparse](https://docs.python.org/zh-cn/3.8/library/argparse.html#module-argparse)

In [ ]:
import argparse

## 1.1 ArgumentParser 对象

class argparse.ArgumentParser(...)
创建一个新的 ArgumentParser 对象。

所有的参数都应当作为关键字参数传入。

### 1.1.1 prog

默认情况下，ArgumentParser 对象使用 `sys.argv[0]` 来确定如何在帮助消息中显示程序名称。

这一默认值几乎总是可取的，因为它将使帮助消息与从命令行调用此程序的方式相匹配。

无论程序从哪个目录下被调用，都这么显示。

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo', help='foo help')
parser.print_help()

可以使用 `prog= 参数` 为 ArgumentParser 指定另一个值

In [ ]:
parser = argparse.ArgumentParser(prog="prog-test")
parser.add_argument('--foo', help='foo help')
parser.print_help()

无论是从 sys.argv[0] 或是从 `prog= 参数` 确定的程序名称，
都可以在帮助消息里通过 %(prog)s 格式说明符来引用。

In [ ]:
parser = argparse.ArgumentParser(prog="prog-test")  # 可以使用 `prog= 参数` 为 ArgumentParser 指定另一个值
parser.add_argument('--foo', help='foo of the %(prog)s program') # 在帮助消息里通过 %(prog)s 格式说明符来引用
parser.print_help()

### 1.1.2 usage
改变帮助信息里的 usage 那一行

In [ ]:
parser = argparse.ArgumentParser(prog='PROG')
parser.add_argument('--foo', nargs='?', help='foo help')
parser.add_argument('bar', nargs='+', help='bar help')
parser.print_help()

In [ ]:
parser = argparse.ArgumentParser(prog='PROG', usage='%(prog)s [options]')
parser.add_argument('--foo', nargs='?', help='foo help')
parser.add_argument('bar', nargs='+', help='bar help')
parser.print_help()

### 1.1.3 description
简要描述这个程序做什么以及怎么做

In [ ]:
# 在默认情况下，description 将被换行以便适应给定的空间。
# 如果想改变这种行为，见 formatter_class 参数
parser = argparse.ArgumentParser(description='A foo that bars')
parser.print_help()

### 1.1.4 epilog
在 description 参数后显示额外的对程序的描述

In [ ]:
# epilog= text 在默认情况下会换行，
# 但是这种行为能够被调整通过提供 formatter_class 参数给 ArgumentParse.
parser = argparse.ArgumentParser(
    description='A foo that bars',
    epilog="And that's how you'd foo a bar")
parser.print_help()

### 1.1.5 parents
通过 `parents= 参数` 让多个解析器使用相同的参数而不是重复这些参数的定义

In [ ]:
# 指定 add_help=False . 
# 否则， ArgumentParse 将会看到两个 -h/--help 选项（一个在父参数中一个在子参数中）
# 并且产生一个错误
parent_parser = argparse.ArgumentParser(add_help=False)
parent_parser.add_argument('--parent', type=int)

foo_parser = argparse.ArgumentParser(parents=[parent_parser])  # 这里的列表
foo_parser.add_argument('foo')
print(foo_parser.parse_args(['--parent', '2', 'XXX']))

bar_parser = argparse.ArgumentParser(parents=[parent_parser])
bar_parser.add_argument('--bar')
print(bar_parser.parse_args(['--bar', 'YYY']))

注解 你在通过``parents=`` 传递解析器之前必须完全初始化它们。 

如果你在子解析器之后改变父解析器，这些改变将不会反映在子解析器上。

### 1.1.6 formatter_class
自定义帮助信息的格式

In [ ]:
import textwrap
parser = argparse.ArgumentParser(
    prog='PROG',
    description=textwrap.dedent('''\
        Please do not mess up this text!
        --------------------------------
            I have indented it
            exactly the way
            I want it
        '''))
parser.print_help()

RawDescriptionHelpFormatter 【仅针对ArgumentParser】

表示 description 和 epilog 已经被正确的格式化了，不能在命令行中被自动换行

In [ ]:
import textwrap
parser = argparse.ArgumentParser(
    prog='PROG',
    formatter_class=argparse.RawDescriptionHelpFormatter,
    description=textwrap.dedent('''\
        Please do not mess up this text!
        --------------------------------
            I have indented it
            exactly the way           
            I want it
        '''))
parser.print_help()

RawTextHelpFormatter 保留所有种类文字的空格，包括参数的描述。 【针对ArgumentParser、add_argument】

然而，多重的新行会被替换成一行。如果你想保留多重的空白行，可以在新行之间加空格。【？】

In [ ]:
import textwrap
parser = argparse.ArgumentParser(
    prog='PROG',
    description=textwrap.dedent('''\
        Please do not mess up this text!
        --------------------------------
            I have indented it
            exactly the way
            I want it
        '''))
parser.add_argument(
    '--bar', 
    help=textwrap.dedent('''
        argparse ArgumentParser
            RawTextHelpFormatter
        '''))
parser.print_help()

In [ ]:
import textwrap
parser = argparse.ArgumentParser(
    prog='PROG',
    formatter_class=argparse.RawTextHelpFormatter,
    description=textwrap.dedent('''\
        Please do not mess up this text!
        --------------------------------
            I have indented it
            exactly the way
            I want it
        '''))
parser.add_argument(
    '--bar', 
    help=textwrap.dedent('''
        argparse ArgumentParser
        
            RawTextHelpFormatter
        '''))
parser.print_help()

ArgumentDefaultsHelpFormatter 自动添加【默认值的信息】到每一个帮助信息的参数中

In [ ]:
parser = argparse.ArgumentParser(
    prog='PROG',
    formatter_class=argparse.ArgumentDefaultsHelpFormatter)
parser.add_argument('--foo', type=int, default=42, help='FOO!')
parser.add_argument('bar', nargs='*', default=[1, 2, 3], help='BAR!')
parser.print_help()

MetavarTypeHelpFormatter 为它的值在每一个参数中使用 type 的参数名当作它的显示名
（而不是使用通常的格式 dest ):

In [ ]:
parser = argparse.ArgumentParser(
    prog='PROG',
    formatter_class=argparse.MetavarTypeHelpFormatter)
parser.add_argument('--foo', type=int)
parser.add_argument('bar', type=float)
parser.print_help()

### 1.1.7 prefix_chars
使用其他符号做前缀

In [ ]:
# prefix_chars 指定的字符串必须包含 `-` 符号
parser = argparse.ArgumentParser(prog='PROG', prefix_chars='-+') 
parser.add_argument('+f')
parser.add_argument('++bar')
parser.add_argument('-b')
print(parser.parse_args('+f X ++bar Y'.split()))
print("-----------------------------")
parser.print_help()

### 1.1.8 fromfile_prefix_chars
把特别长的参数列表存入一个文件中

`fromfile_prefix_chars= 参数`默认为 None，意味着参数不会被当作文件对待

In [ ]:
with open('args.txt', 'w') as fp:
    fp.write('-f\nbar')  # 从文件读取的参数在默认情况下必须一个一行
parser = argparse.ArgumentParser(fromfile_prefix_chars='@')
parser.add_argument('-f')
# 所有类型的字符的参数会被文件包含的参数替代
print(parser.parse_args(['-f', 'foo', '@args.txt']))

### 1.1.9 argument_default【？】
一般情况下，参数默认会通过设置一个默认到 add_argument() 
或者调用带一组指定键值对的 ArgumentParser.set_defaults() 方法。

但是有些时候，为参数指定一个普遍适用的解析器会更有用。

这能够通过传输 argument_default= 关键词参数给 ArgumentParser 来完成。

举个栗子，要全局禁止在 parse_args() 中创建属性，我们提供 argument_default=SUPPRESS:

In [ ]:
parser = argparse.ArgumentParser(argument_default=argparse.SUPPRESS)
parser.add_argument('--foo')
parser.add_argument('bar', nargs='?')
print(parser.parse_args(['--foo', '1', 'BAR']))
print(parser.parse_args([]))

### 1.1.10 allow_abbrev
正常情况下，当你向 ArgumentParser 的 parse_args() 方法传入一个参数列表时，
它会识别缩写。

这个特性可以设置 allow_abbrev 为 False 来关闭。

In [ ]:
parser = argparse.ArgumentParser(prog='PROG')
parser.add_argument('--foobar', action='store_true')
parser.add_argument('--foonley', action='store_false')
print(parser.parse_args(['--foobar','--foon']))

In [ ]:
parser = argparse.ArgumentParser(prog='PROG', allow_abbrev=False)
parser.add_argument('--foobar', action='store_true')
parser.add_argument('--foonley', action='store_false')
print(parser.parse_args(['--foobar','--foon']))

### 1.1.11 conflict_handler
ArgumentParser 对象不允许在 相同选项字符串 下有两种行为

有些时候（例如：使用 parents），重写旧的有相同选项字符串的参数会更有用。

In [ ]:
parser = argparse.ArgumentParser(prog='PROG')
parser.add_argument('-f', '--foo', help='old foo help')
parser.add_argument('--foo', help='new foo help')
parser.print_help()

In [ ]:
parser = argparse.ArgumentParser(prog='PROG', conflict_handler='resolve')
parser.add_argument('-f', '--foo', help='old foo help')
parser.add_argument('--foo', help='new foo help')
parser.add_argument('--foo', help='new new foo help')
parser.print_help()

### 1.1.12 add_help
关闭额外的帮助信息

In [ ]:
parser = argparse.ArgumentParser(prog='PROG', add_help=False)
parser.add_argument('--foo', help='foo help')
parser.print_help()


In [ ]:
parser = argparse.ArgumentParser(prog='PROG')
parser.add_argument('--foo', help='foo help')
parser.print_help()
